## LSTM com Tfidf

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import load_npz

In [2]:
X_train = load_npz('../vectors/tfidf_lemma/X_train_tfidf.npz')
y_train = np.load('../vectors/tfidf_lemma/y_train.npy')

X_test = load_npz('../vectors/tfidf_lemma/X_test_tfidf.npz')
y_test = np.load('../vectors/tfidf_lemma/y_test.npy')

In [3]:
X_train.shape

(6560, 5000)

### Model DNN

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras import layers, models


2023-12-17 13:31:44.954990: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 13:31:45.005898: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 13:31:45.005955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 13:31:45.005989: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 13:31:45.013802: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 13:31:45.014399: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
from scipy.sparse import csr_matrix

In [5]:
dense_matrix = X_train.todense()
X_train_tensor = tf.constant(dense_matrix)

dm = X_test.todense()
X_test_tensor = tf.constant(dm)

2023-12-17 13:31:59.479798: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 262400000 exceeds 10% of free system memory.


In [6]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [8]:
model = models.Sequential()

model.add(layers.Dense(512, activation='relu', input_shape=(5000,)))
model.add(layers.Dropout(0.8))
model.add(layers.Dense(126, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2560512   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 126)               64638     
                                                                 
 dense_2 (Dense)             (None, 4)                 508       
                                                                 
Total params: 2625658 (10.02 MB)
Trainable params: 2625658 (10.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.fit(X_train_tensor, y_train, epochs=10, validation_data=(X_test_tensor, y_test))

Epoch 1/10
205/205 [==============================] - 7s 30ms/step - loss: 1.2823 - accuracy: 0.3931 - val_loss: 1.1880 - val_accuracy: 0.4537
Epoch 2/10
205/205 [==============================] - 6s 30ms/step - loss: 1.0786 - accuracy: 0.5230 - val_loss: 1.1445 - val_accuracy: 0.4780
Epoch 3/10
205/205 [==============================] - 6s 29ms/step - loss: 0.8711 - accuracy: 0.6349 - val_loss: 1.1845 - val_accuracy: 0.4902
Epoch 4/10
205/205 [==============================] - 6s 30ms/step - loss: 0.6779 - accuracy: 0.7416 - val_loss: 1.2723 - val_accuracy: 0.4780
Epoch 5/10
205/205 [==============================] - 6s 30ms/step - loss: 0.5413 - accuracy: 0.7966 - val_loss: 1.3097 - val_accuracy: 0.4835
Epoch 6/10
205/205 [==============================] - 6s 29ms/step - loss: 0.3971 - accuracy: 0.8596 - val_loss: 1.4454 - val_accuracy: 0.4713
Epoch 7/10
205/205 [==============================] - 6s 30ms/step - loss: 0.3248 - accuracy: 0.8816 - val_loss: 1.5207 - val_accuracy: 0.4817

### Model LSTM

In [7]:

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM

In [8]:
max_features = 700
embed_dim = 32
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [9]:
xlstm = []
for text in X_train_tensor:
    ar = np.array(text)
    xlstm.append(ar)

X_train_lstm = pd.DataFrame(xlstm)

In [10]:
xtlstm = []
for text in X_test_tensor:
    ar = np.array(text)
    xtlstm.append(ar)

X_test_lstm = pd.DataFrame(xtlstm)

In [16]:
model = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= X_train_lstm.shape[1]),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
    layers.MaxPooling1D(pool_size=500),
    layers.LSTM(64, dropout=0.4, recurrent_dropout=0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 5000, 32)          22400     
                                                                 
 conv1d_1 (Conv1D)           (None, 5000, 32)          3104      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 10, 32)            0         
 g1D)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 64)                24832     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 4)                 260       
                                                      

In [18]:
history = model.fit(X_train_lstm, y_train, epochs=4, validation_data=(X_test_lstm, y_test))

Epoch 1/4


2023-12-17 14:39:08.730648: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 262400000 exceeds 10% of free system memory.


205/205 [==============================] - 14s 59ms/step - loss: 0.5564 - accuracy: 0.3587 - val_loss: 0.5479 - val_accuracy: 0.3659
Epoch 2/4
205/205 [==============================] - 12s 58ms/step - loss: 0.5471 - accuracy: 0.3659 - val_loss: 0.5489 - val_accuracy: 0.3659
Epoch 3/4
205/205 [==============================] - 15s 74ms/step - loss: 0.5472 - accuracy: 0.3659 - val_loss: 0.5462 - val_accuracy: 0.3659
Epoch 4/4
205/205 [==============================] - 15s 74ms/step - loss: 0.5467 - accuracy: 0.3659 - val_loss: 0.5469 - val_accuracy: 0.3659


In [12]:
model2 = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= X_train_lstm.shape[1]),
    #layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
    #layers.MaxPooling1D(pool_size=2),
    layers.LSTM(64, dropout=0.4, recurrent_dropout=0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5000, 32)          22400     
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 49444 (193.14 KB)
Trainable params: 49444 (193.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
history2 = model2.fit(X_train_lstm, y_train, epochs=5, validation_data=(X_test_lstm, y_test))

Epoch 1/5


2023-12-17 13:33:05.591611: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 262400000 exceeds 10% of free system memory.


205/205 [==============================] - 779s 4s/step - loss: 0.5575 - accuracy: 0.3582 - val_loss: 0.5472 - val_accuracy: 0.3659
Epoch 2/5
205/205 [==============================] - 790s 4s/step - loss: 0.5472 - accuracy: 0.3659 - val_loss: 0.5468 - val_accuracy: 0.3659
Epoch 3/5
205/205 [==============================] - 779s 4s/step - loss: 0.5472 - accuracy: 0.3659 - val_loss: 0.5463 - val_accuracy: 0.3659
Epoch 4/5
205/205 [==============================] - 762s 4s/step - loss: 0.5469 - accuracy: 0.3659 - val_loss: 0.5462 - val_accuracy: 0.3659
Epoch 5/5
205/205 [==============================] - 781s 4s/step - loss: 0.5465 - accuracy: 0.3659 - val_loss: 0.5472 - val_accuracy: 0.3659
